# Performance des investissements

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import yaml

In [ ]:
with open("config.yaml", "r") as f:
    config = yaml.safe_load(f)

INTERET_LIVRET_A = config["INTERET_LIVRET_A"]
DATA_FOLDER = config["DATA_FOLDER"]
INFLATION = config["INFLATION"]
IMAGE_FOLDER = config["IMAGE_FOLDER"]
TMP_FOLDER = config["TMP_FOLDER"]

In [ ]:
def calcul_pmv(row):
    if row["Type"] != "livret":
        return row["Valorisation"] - row["Apport"]
    else:
        return 0


def calcul_perf(row):
    if row["Type"] != "livret":
        return row["Plus-Moins Value"] / row["Valorisation"]
    else:
        return INTERET_LIVRET_A


def calcul_apport(df, actif):
    return df.loc[df["Actif"] == actif, "Montant"].sum()


def color_based_on_value(val):
    if val > INFLATION * 2:
        return "background-color: lightgreen"
    elif val > INFLATION:
        return "background-color: gold"
    else:
        return "background-color: salmon"


def my_print(df):
    return (
        df.drop(columns=["Type"])
        .style.format(
            {
                "Valorisation": "{:,.2f}€",
                "Apport": "{:,.2f}€",
                "Plus-Moins Value": "{:,.2f}€",
                "Performance": "{:.2%}",
            }
        )
        .hide()
    ).map(color_based_on_value, subset=["Performance"])

In [ ]:
apport = pd.read_csv(DATA_FOLDER + "apport.csv")
apport["Date"] = pd.to_datetime(apport["Date"], dayfirst=True)
apport["Montant"] = pd.to_numeric(apport["Montant"])

In [ ]:
patrimoine = pd.read_csv(DATA_FOLDER + "valorisation.csv")
patrimoine["Valorisation"] = pd.to_numeric(patrimoine["Valorisation"])
patrimoine = patrimoine.sort_values(
    by=["Valorisation"], ascending=False, ignore_index=True
)
apport_list = []
for actif in patrimoine["Actif"]:
    apport_list.append(calcul_apport(apport, actif))
patrimoine["Apport"] = apport_list

In [ ]:
patrimoine["Plus-Moins Value"] = patrimoine.apply(calcul_pmv, axis=1)
patrimoine["Performance"] = patrimoine.apply(calcul_perf, axis=1)

In [ ]:
perf_total = (
    patrimoine.loc[patrimoine["Type"] != "livret", "Plus-Moins Value"].sum()
    / patrimoine.loc[patrimoine["Type"] != "livret", "Valorisation"].sum()
)

pmv_total = patrimoine.loc[patrimoine["Type"] != "livret", "Plus-Moins Value"].sum()

patrimoine.to_csv(TMP_FOLDER + "patrimoine.csv", encoding="utf-8")
my_print(patrimoine)

In [ ]:
normal_print = lambda x: "{:,.2f}€".format(x)
plus_value_clean = patrimoine["Plus-Moins Value"].apply(normal_print)

plt.figure(figsize=(10, 6))
barplot = sns.barplot(data=patrimoine, x="Actif", y="Performance", hue=plus_value_clean)

for index, row in patrimoine.iterrows():
    # Afficher la performance avec 2 décimales
    plt.text(
        index,
        row["Performance"],  # Position Y légèrement au-dessus de la barre
        f"{row['Performance']*100:.2f} %",  # Texte avec formatage à 2 décimales
        ha="center",
        va="bottom",
        fontsize=10,
        color="black",
    )

plt.text(
    2,
    -0.05,
    f"Plus/Moins value total : {pmv_total:,.2f}€\nPerformance total : {perf_total:.2%}",
    ha="center",
    fontsize=12,
    color="black",
)

plt.title("Pérformance par Actif", fontsize=14)
plt.xlabel("Actif")
plt.ylabel("Pérformance")
plt.legend(title="Plus/Moins value")
plt.tight_layout()

plt.savefig(IMAGE_FOLDER + "perf-par-actif.png")
plt.close()